[From Bob Nasuti, grab your wallet and take a seat:](https://fivethirtyeight.com/features/hark-two-holiday-puzzles/)

Consider a game of chance called Left, Right, Center. Everyone sits in a circle and begins with some \$1 bills. Taking turns, each person rolls three dice. For each die, if you roll a 1 or 2 you give a dollar to the person on your left, if you roll a 3 or 4 you give a dollar to the person on your right, and if you roll a 5 or 6 you put a dollar in the middle. The moment only a single person has any money left, the game ends and that person gets all the money in the center.

How long is the game expected to last for six players each starting with three \$1 bills? For X players each starting with Y $1 bills?

In [229]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [18]:
outcomes_ = {1:'$1 to left',2:'$1 to left',3:'$1 to right',4:'$1 to right',5:'$1 to middle',6:'$1 to middle'}
outcomes_update = {1:[1,-1,0],2:[1,-1,0],3:[0,-1,1],4:[0,-1,1],5:[0,-1,0],6:[0,-1,0]}
def dice():
    return np.random.choice(np.arange(1,7),size=1,replace=False)[0]

# 6 players starting with three \$1 bills

In [208]:
def one_round(initial,count,print_ = False):
    for ii,player in enumerate(initial):
        left = ii - 1 if (ii - 1 > -1) else len(initial)-1
        right = ii + 1 if (ii + 1 <len(initial)) else 0
        update_ = initial
        #print(update_)
        for _ in range(3):
            outcome = outcomes_update[dice()]
            count += 1
            if print_:
                print([left,ii,right],outcome)
            if update_[ii] + outcome[1] >=0:
                update_[ii] = update_[ii] + outcome[1]
                update_[left] = update_[left] + outcome[0]
                update_[right] = update_[right] + outcome[-1]
                if print_:
                    print(update_)
            else:
                pass
        initial = update_
        if len(np.nonzero(initial)[0])<=1:
            #print(initial)
            return initial,count
            break
    else:
        return initial,count

In [224]:
results = []
simulate = int(1e4)
for _ in range(simulate):
    initial = np.array([m for _ in range(n)])
    count = 0
    for _ in range(90):
        initial,count=one_round(initial,count)
        if len(np.nonzero(initial)[0])<=1:
            break
    
    results.append(count)

In [225]:
print('%d players each with $%d,%.4f+/-%.4f'%(m,n,np.mean(results)/3,np.std(np.array(results)/3)))

3 players each with $3,6.8727+/-2.7991


In [ ]:
n_players = np.arange(3,10)
n_bills = np.arange(3,15)
simulate = int(1e4)
results = {}
for m in n_players:
    for n in n_bills:
        results[(m,n)] = []
        for _ in tqdm(range(simulate),desc='simulate'):
            initial = np.array([m for _ in range(n)])
            count = 0
            for _ in range(int(1e6)):
                initial,count = one_round(initial,count)
                if len(np.nonzero(initial)[0]) <=1:
                    #print(n,m,'break',end='')
                    break
            results[(m,n)].append(count)

simulate:  16%|████▎                     | 1646/10000 [00:05<00:27, 303.91it/s]